In [9]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

In [13]:
project_root = Path("/home/justin/projects/fight_predictor")
data_dir = project_root / "predictor" / "Data" / "Scraped_Data"

fighters = pd.read_csv(data_dir / "scraped_fighters.csv")

In [25]:
fighters.dtypes

date_of_birth      object
fighter_name       object
fighter_record     object
height             object
reach             float64
sapm              float64
slpm              float64
stance             object
strike_acc          int64
strike_def          int64
sub_avg           float64
td_acc              int64
td_avg            float64
td_def              int64
weight            float64
dtype: object

In [15]:
bouts = pd.read_csv(data_dir / "scraped_bouts.csv")

In [24]:
bouts.head()

,event_attendance,event_date,event_name,fighter1,fighter2,pass_stat_f1,pass_stat_f2,round,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2,time,weight_class,win_method_finish,win_method_type,winner
0,18117,June 09 2018,UFC 225: Whittaker vs. Romero 2,Robert Whittaker,Yoel Romero,0,0,5,128,111,0,0,0,3,5.0,Middleweight,NaN,S-DEC,Robert Whittaker
1,18117,June 09 2018,UFC 225: Whittaker vs. Romero 2,Colby Covington,Rafael Dos Anjos,2,1,5,99,105,0,0,7,3,5.0,Welterweight,NaN,U-DEC,Colby Covington
2,18117,June 09 2018,UFC 225: Whittaker vs. Romero 2,Holly Holm,Megan Anderson,9,0,3,47,26,1,0,4,0,5.0,Women's Featherweight,NaN,U-DEC,Holly Holm
3,18117,June 09 2018,UFC 225: Whittaker vs. Romero 2,Tai Tuivasa,Andrei Arlovski,1,0,3,54,51,0,0,0,0,5.0,Heavyweight,NaN,U-DEC,Tai Tuivasa
4,18117,June 09 2018,UFC 225: Whittaker vs. Romero 2,Mike Jackson,CM Punk,1,1,3,64,19,0,0,1,1,5.0,Welterweight,NaN,U-DEC,Mike Jackson


In [26]:
bouts.dtypes

event_attendance       int64
event_date            object
event_name            object
fighter1              object
fighter2              object
pass_stat_f1           int64
pass_stat_f2           int64
round                  int64
str_stat_f1            int64
str_stat_f2            int64
sub_stat_f1            int64
sub_stat_f2            int64
td_stat_f1             int64
td_stat_f2             int64
time                 float64
weight_class          object
win_method_finish     object
win_method_type       object
winner                object
dtype: object

In [31]:
bouts["event_date"] = pd.to_datetime(bouts["event_date"])
bouts.sort_values(by="event_date", ascending=False)

,event_attendance,event_date,event_name,fighter1,fighter2,pass_stat_f1,pass_stat_f2,round,str_stat_f1,str_stat_f2,sub_stat_f1,sub_stat_f2,td_stat_f1,td_stat_f2,time,weight_class,win_method_finish,win_method_type,winner
55,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Alejandro Perez,Eddie Wineland,0,0,3,53,48,0,0,0,0,5.00,Bantamweight,NaN,U-DEC,Alejandro Perez
54,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Cat Zingano,Marion Reneau,2,0,3,33,14,0,0,6,0,5.00,Women's Bantamweight,NaN,U-DEC,Cat Zingano
53,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Chad Mendes,Myles Jury,0,0,1,17,3,0,0,0,0,2.52,Featherweight,Punch,KO/TKO,Chad Mendes
52,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Niko Price,Randy Brown,0,1,2,12,13,0,0,1,1,1.09,Welterweight,Punch,KO/TKO,Niko Price
51,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Rick Glenn,Dennis Bermudez,0,1,3,48,42,1,0,0,6,5.00,Featherweight,NaN,S-DEC,Rick Glenn
50,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Sage Northcutt,Zak Ottow,1,6,2,24,6,1,1,1,1,3.13,Welterweight,Punch,KO/TKO,Sage Northcutt
49,5648,2018-07-14,UFC Fight Night: Dos Santos vs. Ivanov,Junior Dos Santos,Blagoy Ivanov,0,0,5,110,51,0,0,0,0,5.00,Heavyweight,NaN,U-DEC,Junior Dos Santos
42,17464,2018-07-07,UFC 226: Miocic vs. Cormier,Khalil Rountree Jr.,Gokhan Saki,0,0,1,9,5,0,0,0,0,1.36,Light Heavyweight,Punches,KO/TKO,Khalil Rountree Jr.
38,17464,2018-07-07,UFC 226: Miocic vs. Cormier,Daniel Cormier,Stipe Miocic,0,0,1,25,24,0,0,0,1,4.33,Heavyweight,Punches,KO/TKO,Daniel Cormier
39,17464,2018-07-07,UFC 226: Miocic vs. Cormier,Derrick Lewis,Francis Ngannou,0,0,3,20,11,0,0,0,0,5.00,Heavyweight,NaN,U-DEC,Derrick Lewis


In [32]:
len(bouts)

56

In [1]:
from preprocess_data import FightDataPreprocessor
import tensorflow.keras as tf

ModuleNotFoundError: No module named 'preprocess_data'

In [2]:
fdp = FightDataPreprocessor()


In [3]:
fighters = fdp.get_fighters()

/Users/charlespierse/PycharmProjects/UFC_Fight_Predictor/Fight_Predictor
/Users/charlespierse/PycharmProjects/UFC_Fight_Predictor/Fight_Predictor/Fight_Predictor/Fights_scraper/spiders/scraped_fighters.csv


SystemExit: Unable to read Fighters file from disk. Exiting.

/Users/charlespierse/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
fighters.head()

,date_of_birth,fighter_name,fighter_record,height,reach,sapm,slpm,stance,strike_acc,strike_def,sub_avg,td_acc,td_avg,td_def,weight
0,Jun 25 1981,Luciano Azevedo,Record: 16-9-1,"6' 3""",NaN,1.97,0.76,Orthodox,45,27.0,0.0,11,2.28,0,161.0
1,Jul 17 1990,Michael Byrnes,Record: 5-2-0,"5' 11""",NaN,0.00,0.00,NaN,0,0.0,0.0,0,0.00,0,155.0
2,Feb 07 1989,Gleidson Cutis,Record: 7-3-0,"5' 9""",NaN,8.28,2.99,Orthodox,52,59.0,0.0,0,0.00,0,155.0
3,Aug 11 1990,Rolando Dy,Record: 9-7-1 (1 NC),"5' 8""",69.0,4.47,3.04,Orthodox,37,52.0,0.0,20,0.30,68,145.0
4,Jul 27 1986,Jessica Eye,Record: 14-6-0 (1 NC),"5' 6""",66.0,3.43,3.63,Orthodox,36,60.0,0.7,50,0.53,60,125.0


In [11]:
def get_fighter_details(fighter_name):
    return fighters[fighters.fighter_name==fighter_name]
    

In [12]:
get_fighter_details('Conor McGregor')
get_fighter_details('Anthony Pettis')

,date_of_birth,fighter_name,fighter_record,height,reach,sapm,slpm,stance,strike_acc,strike_def,sub_avg,td_acc,td_avg,td_def,weight
87,Jan 27 1987,Anthony Pettis,Record: 22-8-0,"5' 10""",72.0,3.22,2.87,Orthodox,45,54.0,1.2,54,0.73,58,170.0


In [18]:
import os

os.getcwd()

'/Users/charlespierse/PycharmProjects/UFC_Fight_Predictor/Fight_Predictor'

In [20]:
tf.models.load_model(os.getcwd()+'/Saved_Models/Winner_Prediction_Models/bout_winner.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [ ]:
def pass_fighters_through_pipeline(fighter1,fighter2):
    